In [21]:
# pip install /root/CodeCommit/msspackages/dist/msspackages-0.0.7-py3-none-any.whl

In [ ]:
# from msspackages import setup_runner
# setup_runner()

In [22]:
from feature_engineering import node_autoencoder_ad_preprocessing, node_autoencoder_ad_feature_engineering, node_autoencoder_train_test_split
from feature_engineering import container_autoencoder_ad_preprocessing, container_autoencoder_ad_feature_engineering, container_autoencoder_train_test_split
from feature_engineering import pod_autoencoder_ad_preprocessing, pod_autoencoder_ad_feature_engineering, pod_autoencoder_train_test_split

from utilities import s3_operations
import pandas as pd

In [23]:
def training_data_builder(preprocessing_args,
                          preprocessing_fnc, 
                          preprocessing_train_test_split_fnc,
                          feature_engineering_fnc):
    
    #pre processing
    features_data, processed_data = preprocessing_fnc(*preprocessing_args)
    #pd.set_option('display.max_columns', None)  
    print(features_data.head())
    print(processed_data.show(truncate=False))
    
    #test, train split
    train_data, test_data = preprocessing_train_test_split_fnc(processed_data)

    #Train data feature engineering
    training_data = feature_engineering_fnc(features_data, train_data)
    #s3_operations.write_to_s3(node_training_data, s3_path)
    print(training_data.show(truncate=False))
 
    #Test data feature engineering
    testing_data = feature_engineering_fnc(features_data, test_data)
    #s3_operations.write_to_s3(node_testing_data, s3_path)
    print(testing_data.show(truncate=False))
    
    return training_data, testing_data

def pod_training_data_builder():
    
    #pre processing
    pod_features_data, pod_processed_data = pod_autoencoder_ad_preprocessing("pod_autoencoder_ad","11-30-2022","2022","10","10","10")
    #pd.set_option('display.max_columns', None)  
    print(pod_features_data.head())
    print(pod_processed_data.show(truncate=False))
    
    #test, train split
    pod_train_data, pod_test_data = pod_autoencoder_train_test_split(pod_processed_data)

    #Train data feature engineering
    pod_training_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_train_data)
    #s3_operations.write_to_s3(node_training_data, s3_path)
    print(pod_training_data.show(truncate=False))
 
    #Test data feature engineering
    pod_testing_data = pod_autoencoder_ad_feature_engineering(pod_features_data, pod_test_data)
    #s3_operations.write_to_s3(node_testing_data, s3_path)
    print(pod_testing_data.show(truncate=False))
    
    return pod_training_data, pod_testing_data

In [ ]:
#build training data for pod
pod_training_data, pod_testing_data =training_data_builder(preprocessing_args = ["pod_autoencoder_ad","11-30-2022","2022","10","10","10"],
                                                           preprocessing_fnc = autoencoder_ad_preprocessing, 
                                                           preprocessing_train_test_split_fnc = pod_autoencoder_train_test_split,
                                                           feature_engineering_fnc = pod_autoencoder_ad_feature_engineering)